In [2]:
import random

import torch 
import numpy as np
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [3]:
text_chunks_and_embeddings = pd.read_csv('C:\\Users\\hp probook\\Desktop\\RAG_Project\\data_preprocessing\\bdd.csv')

text_chunks_and_embeddings['embeddings'] = text_chunks_and_embeddings['embeddings'].apply(lambda x : np.fromstring(x.strip("[]"), sep=', '))

# text_chunks_and_embeddings['embeddings'] = text_chunks_and_embeddings['embeddings'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

pages_and_chunks = text_chunks_and_embeddings.to_dict(orient= "records")
pages_and_chunks

[{'page_number': 1,
  'sentence_chunk': '1  Préambule  Après onze années en tant que chargé de cours du module « Bases De Données » à  l’Ecole Supérieure d’Informatique, je me rappelle toujours du coup de fil de Mr Medjaoui  (Directeur des études de l’époque) me demandant d’assurer ce cours pour les étudiants de  3ème année option « Systèmes d’Information ». A peine recruté, je me retrouve comme  chargé de cours devant 150 étudiants, tous assoiffés de savoir et qui en demandaient toujours  d’avantage. N’ayant pas une grande expérience en pédagogie, ce fut uniquement mon expérience  professionnelle en tant que développeur et administrateur de bases de données qui a poussé  le DE à me désigner pour cette tâche. J’ai commencé alors à chercher des supports de cours  pour m’imprégner du module et préparer mes leçons, mais, hélas, je n’ai pas trouvé ne serait- ce qu’un brouillon. Je me suis alors réfugié à la bibliothèque de l’école où j’ai pratiquement  dévoré tous les livres qui parlaient 

In [4]:
embeddings = torch.tensor(np.stack(text_chunks_and_embeddings['embeddings'].tolist(), axis=0))
embeddings.shape

torch.Size([165, 768])

In [5]:
from sentence_transformers import SentenceTransformer, util

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device= device)

### steps:
* define a query string 
* turn the query into an embedding vector 
* perform the similarity search by using teh cosine similarity function between the chunks embedding and the query embedding
* choose the 5 chunks with highest degree of similarity   

In [5]:
"""
faiss library to create vector database
and ann methode to seacrh the relevent passages 
cosine similarity is just the dot product of values already normalized (L2 norm : la racine de (la somme des ( valeurs au carré))), and it's favored to use cosine similarity then the dot product when we do semantic search

mixedbread rerank model used to rerank the top k values 
falshattention to speed up the attention mechanism of transformers and generating the token of our llm
"""

"\nfaiss library to create vector database\nand ann methode to seacrh the relevent passages \ncosine similarity is just the dot product of values already normalized (L2 norm : la racine de (la somme des ( valeurs au carré))), and it's favored to use cosine similarity then the dot product when we do semantic search\n\nmixedbread rerank model used to rerank the top k values \nfalshattention to speed up the attention mechanism of transformers and generating the token of our llm\n"

In [6]:
#define a query 
query = "explique la théorie de la normalisation"
print(f"query: {query}")

embeded_query = embedding_model.encode(query, convert_to_tensor=True).to('cpu')
embeded_query

from time import perf_counter as timer 

dot_scores = util.dot_score(a=embeded_query.float(), b=embeddings.float())[0]

top_values = torch.topk(dot_scores, k=5)

print(top_values)


query: explique la théorie de la normalisation
torch.return_types.topk(
values=tensor([0.7604, 0.5894, 0.5825, 0.5562, 0.5499]),
indices=tensor([104, 110, 105, 109, 108]))


In [7]:
for value, index in zip(top_values.values.tolist(), top_values.indices.tolist()):
    print(f"score:  {value}")
    print(f"text: {pages_and_chunks[index]['sentence_chunk']}")
    print(f"page:  {pages_and_chunks[index]['page_number']}")
    print('\n')
    # print(f"value : {value}")
    # print(f"index : {index}")

score:  0.7603526711463928
text: Théorie de la normalisation  Cette théorie est basée sur les dépendances fonctionnelles qui permettent de  décomposer l'ensemble des informations en diverses relations. Chaque nouvelle forme  normale marque une étape dans la progression vers des relations présentant de moins en  moins de redondance. On applique les formes normales successivement à la relation  universelle afin d’obtenir un schéma normalisé.
page:  93


score:  0.5893518328666687
text: Chapitre 3 : Le modèle Relationnel    99  La troisième forme normale ajoute une autre restriction à la seconde forme normale  en exprimant le fait que tous les attributs non clé dépendent complètement et uniquement  de la clé de la relation. La Boyce Codd forme normale pousse plus loin la restriction de la troisième forme  normale en exprimant le fait que dans toute relation en deuxième forme normale, s'il existe  une dépendance fonctionnelle alors sa partie gauche est forcément une clé de cette relation. 

### Functionazing the semantic search pipeline:

In [6]:
def retrieval_semantic_search(query: str,
                    embeddings: torch.tensor,
                    top_n: int,
                    model: SentenceTransformer= embedding_model):
    
    query_embeded = model.encode(query,convert_to_tensor=True)

    #model encode return embeddings already normalized
    dot_scores = util.dot_score(query_embeded.float(),embeddings.float())
    top_scores_indexs = torch.topk(dot_scores, k=top_n)

    return top_scores_indexs

def top_scores_indexs(query: str,
                      embeddings: torch.tensor,
                      top_n: int,
                      model: SentenceTransformer= embedding_model):

    top_scores_indexs = retrieval_semantic_search(query,
                          embeddings = embeddings,
                          top_n=top_n)
    return_list = []
    #printing them
    for value, index in zip(top_scores_indexs.values.tolist()[0], top_scores_indexs.indices.tolist()[0]):
        values_map = {
            "score": value,
            "text": pages_and_chunks[index]['sentence_chunk'],
            "page":  pages_and_chunks[index]['page_number']
        }
        return_list.append(values_map)
    return return_list

In [9]:
query = "explique la théorie de la normalisation"
top_scores_indexs(query=query,
                  embeddings = embeddings,
                  top_n=5)

[{'score': 0.7603526711463928,
  'text': "Théorie de la normalisation  Cette théorie est basée sur les dépendances fonctionnelles qui permettent de  décomposer l'ensemble des informations en diverses relations. Chaque nouvelle forme  normale marque une étape dans la progression vers des relations présentant de moins en  moins de redondance. On applique les formes normales successivement à la relation  universelle afin d’obtenir un schéma normalisé.",
  'page': 93},
 {'score': 0.5893518328666687,
  'text': "Chapitre 3 : Le modèle Relationnel    99  La troisième forme normale ajoute une autre restriction à la seconde forme normale  en exprimant le fait que tous les attributs non clé dépendent complètement et uniquement  de la clé de la relation. La Boyce Codd forme normale pousse plus loin la restriction de la troisième forme  normale en exprimant le fait que dans toute relation en deuxième forme normale, s'il existe  une dépendance fonctionnelle alors sa partie gauche est forcément une 

### Modeling:

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers.utils import is_flash_attn_2_available 

# quantization_config = BitsAndBytesConfig(load_in_4bit=True,
#                                          bnb_4bit_compute_dtype=torch.float16)
model_id = 'google/gemma-2b-it' 
print(f"model_id: {model_id}")

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)

llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id, 
                                                 torch_dtype=torch.bfloat16) # which attention version to use

model_id: google/gemma-2b-it


Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
llm_model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRMSNo

In [12]:
def get_model_num_params(model: torch.nn.Module):
    return sum([param.numel() for param in model.parameters()])

get_model_num_params(llm_model)

2506172416

In [13]:
def get_model_mem_size(model: torch.nn.Module):
    # Get model parameters and buffer sizes
    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])

    # Calculate various model sizes
    model_mem_bytes = mem_params + mem_buffers # in bytes
    model_mem_mb = model_mem_bytes / (1024**2) 
    model_mem_gb = model_mem_bytes / (1024**3) 

    return {"model_mem_bytes": model_mem_bytes,
            "model_mem_mb": round(model_mem_mb, 2),
            "model_mem_gb": round(model_mem_gb, 2)}

get_model_mem_size(llm_model)

{'model_mem_bytes': 5012344832, 'model_mem_mb': 4780.14, 'model_mem_gb': 4.67}

In [14]:
input_text = "explique la théorie de la normalisation"
print(f"Input text:\n{input_text}")

#craete prompt template for instruction-tuned model
dialogue_template = [
    {"role": "user",
     "content": input_text}
]

#apply the chat template
prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                       tokenize=False, # keep it not tokenized
                                       add_generation_prompt=True)
print(f"\nPrompt (formatted):\n{prompt}")

Input text:
explique la théorie de la normalisation

Prompt (formatted):
<bos><start_of_turn>user
explique la théorie de la normalisation<end_of_turn>
<start_of_turn>model



In [15]:
%%time

inputs = tokenizer.encode(prompt, 
                      add_special_tokens= True,
                      return_tensors="pt" )

outputs = llm_model.generate(input_ids=inputs.to(llm_model.device), max_new_tokens=150)

CPU times: total: 2min 29s
Wall time: 1min 30s


In [16]:
outputs[0]

tensor([     2,      2,    106,   1645,    108, 214032,    683, 117577,    581,
           683,   4500,   5076,    107,    108,    106,   2516,    108,    688,
          2841, 117577,    581,    683,   4500,   5076,    688,   1455,   2360,
        117577, 160719,   2459,  22244,    581,   4500,   5975,   1437,  26173,
          2173, 166333,  22424,   4255, 235265,  46531,  72828,    700, 235303,
           477,   9273,   1437,  46895,    659,  26173, 166333,  22424,   4255,
        235269,   2906,   2459, 121536,   1437,  16876, 128648,   1008,    683,
        164526,   3633,    848,  26173, 166333,  22424,   4255, 235265,    109,
           688,  13274,    667,  63245,  87995,    581,    683,   4500,   5076,
           865,    688,    109, 235287,   5231,  83057,    581,    683,   8792,
           865,    688,   2221,   8792,   4500,  47529,   1455,  55360,    547,
          7077,    683,  39933,    581,    683,   8792,  38304,  36395,   2461,
           755,   2360,  39933,  51391, 

In [17]:
#convert the output tokens into readable text
output_decoded = tokenizer.decode(outputs[0])
print(output_decoded)

<bos><bos><start_of_turn>user
explique la théorie de la normalisation<end_of_turn>
<start_of_turn>model
**La théorie de la normalisation** est une théorie statistique qui permet de normaliser les données non normalement distribuées. Cela signifie qu'on peut les transformer en données normalement distribuées, ce qui facilite les analyses statistiques et la comparaison avec des données normalement distribuées.

**Principaux principes de la normalisation :**

* **Transformation de la variable :** La variable normalisée est définie comme la valeur de la variable originale divisée par une valeur moyenne et une valeur standard deviation.
* **Transformation inverse :** La valeur de la variable normale est obtenue en multipliant la valeur de la variable originale par la valeur moyenne et en ajoutant la valeur standard deviation.

**Applications de la normalisation :**

*


In [8]:
import pandas as pd

context_data = pd.read_csv('C:\\Users\\hp probook\\Desktop\\RAG_Project\\data_preprocessing\\bdd.csv')
context_data

,page_number,sentence_chunk,tokens_number,embeddings
0,1,1 Préambule Après onze années en tant que ch...,94.00,"[0.008394155651330948, -0.051100436598062515, ..."
1,1,J’ai alors fait des diapos pour chaque chapitr...,46.75,"[-0.004756780341267586, -0.0710226446390152, -..."
2,2,"2 Pour finir, je pense avoir synthétisé ma mo...",12.75,"[0.031454432755708694, 0.0174418855458498, -0...."
3,13,13 Présentation du cours Ce cours s’adresse ...,77.50,"[-0.0037500502075999975, -0.030021093785762787..."
4,13,"Théorie des ensembles, 3. Logique du premier ...",19.50,"[-0.02807949110865593, 0.009503806941211224, -..."
...,...,...,...,...
160,145,145 Annexe 1 : Les Fonctions dans SQL 92 Do...,69.25,"[-0.013162732124328613, -0.02656504325568676, ..."
161,146,146 IS [NOT] NULL NULL INNER JOIN Jointure...,28.50,"[-0.04927458614110947, -0.01684270054101944, -..."
162,147,147 Annexe 2 : Outils Pédagogiques 1. Functi...,45.25,"[-0.01167414989322424, 0.0013812012039124966, ..."
163,147,Il est utilisé par les étudiants pour confront...,12.75,"[-0.004473675042390823, -0.018602140247821808,..."


* Augmenting the prompt with the context items

In [9]:
def prompt_augment(query: str,
                   context_items: list[dict]):
    context = "- " + "\n- ".join([item['text'] for item in context_items])

    base_prompt ="""based on these context items, please answer the question:
    context_items: {context}
    query: {query}
    Answer: 
    """    
    base_prompt = base_prompt.format(context=context,
                                query = query)
    # input_text = context + "\n\n" + query
    
    dialogue_template = [
    {"role": "user",
     "content": base_prompt}
    ]
    
    #apply the chat template
    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                           tokenize=False, # keep it not tokenized
                                           add_generation_prompt=True)
    return prompt

In [30]:
query ="explique la théorie de la normalisation"
context = top_scores_indexs(query=query,
                          embeddings = embeddings,
                          top_n=5)
prompt = prompt_augment(query, context)

In [31]:
%%time
inputs = tokenizer.encode(prompt, 
                      add_special_tokens= True,
                      return_tensors="pt" )

outputs = llm_model.generate(input_ids=inputs,
                            temperature = 0.7,
                            max_new_tokens=256)

c:\Users\hp probook\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


CPU times: total: 9min 33s
Wall time: 6min 24s


In [ ]:
#convert the output tokens into readable text
output_decoded = tokenizer.decode(outputs[0])

print(f"Query: {query}")
print(f"Answer:\n{output_decoded.replace(prompt,' ')}")

Query: explique la théorie de la normalisation
Answer:
<bos> **La théorie de la normalisation** est une approche pour améliorer les performances des bases de données en minimisant les redundances et en garantissant la validité des données.

**Les trois formes normales** sont :

1. **Première forme normale (1NF)** : une relation est en 1NF si tous les attributs sont de type simple (non-multiples et non-composés).
2. **Deuxième forme normale (2NF)** : une relation est en 2NF si toutes les relations non clés sont dépendantes de la clé.
3. **Troisième forme normale (3NF)** : une relation est en 3NF si toutes les relations non clés sont dépendantes de la clé et que la clé est minimale (une clé minimale est une clé qui ne contient aucune autre clé comme dépendance).

**La normalisation** vise à atteindre une relation en 3NF en isolant les dépendances fonctionnelles de la clé et en exprimant ces dépendances de manière plus concise. Cela permet de réduire les redundances et de garantir la vali

### Functionize all the process from thge query to the answer generation:

- takes a query
- find relevent contexts 
- augment the prompt
- tokenize it
- generate the answer

In [10]:
def ask_dahak(query: str, 
              temperature: float=0.7, 
              max_new_token: int=256,
              format_answer_text = True,
              answer_only = True
               ):

    context = top_scores_indexs(query=query,
                                embeddings = embeddings,
                                top_n=5)
    
    prompt = prompt_augment(query=query, context_items=context)
    
    inputs = tokenizer.encode(prompt, 
                          add_special_tokens= True,
                          return_tensors="pt" )
    
    outputs = llm_model.generate(input_ids=inputs,
                                temperature = temperature,
                                max_new_tokens=max_new_token)

    output_text = tokenizer.decode(outputs[0])
    
    # format the answer 
    if format_answer_text:
        # replace the tokens and remove the prompt
        output_text = output_text.replace(prompt," ").replace("<bos>", " ").replace("<eos>"," ")
    
    if answer_only:
        return output_text #without the context items
    
    
    return output_text, context

In [13]:
query = "explique l'entité faible et donne moi exemple"
print(f"Query: {query}")
answer, context = ask_dahak(query, answer_only=False)

print(f"Answer: {answer}")

print(f"\nContext: {context}")

Query: explique l'entité faible et donne moi exemple


c:\Users\hp probook\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Answer:   **Entité faible**

Une entité faible est une entité possédant un identifiant insuffisant de par lui-même pour identifier de manière unique chacune de ses occurrences. Sa caractéristique d’identifiant n’est valable qu’à l’intérieur du contexte spécifique de l’occurrence d’une entité principale.

**Exemple**

Dans notre exemple, l'identifiant d'une chambre est constitué de deux parties : l'identifiant de l'hôtel et le numéro de la chambre. 

Context: [{'score': 0.5391520857810974, 'text': ' On peut dire finalement, qu’une entité est une réalisation ou concrétisation des  propriétés d’un type-entité ce qui signifie qu’on donne une valeur à chaque attribut du type- entité pour obtenir une entité de ce type-entité.', 'page': 24}, {'score': 0.4777226746082306, 'text': ' La relation est représentée sous une forme tabulaire dont chaque ligne représente une  extension.', 'page': 79}, {'score': 0.46703875064849854, 'text': "Chapitre 2 : Conception des bases de données avec le modèle En